# DEMO: runOvertopping.exe

In [ ]:
from probabilistic_library import ReliabilityProject, DistributionType, CompareType, ReliabilityMethod
from ptk import Model, Template # wrapper that supports connection of models
import os

### Model

In [ ]:
model = Model()
model.executable_file = "../models/dikesOvertopping/runOvertopping.exe"
model.input_file = "../models/overtopping.txt"
model.output_file = "{name}.out"
model.arguments = '{input} >{name}.out'

if os.path.exists(model.executable_file):
    print (model.executable_file + " exists")
else:
    print (model.executable_file + " does not exist!")

### Input file

In [ ]:
h_template = Template()
model.input_templates.append(h_template)

print("available captions: ")
for caption in h_template.get_captions():
    print (caption)

h_template.caption = "Row 7"
print("available attributes: ")
for attribute in h_template.get_attributes():
    print (attribute)

### Templates

In [ ]:
h_template.caption_name = "load"
h_template.attribute = "Column 1"
h_template.attribute_name = "h"

hs_template = Template()
hs_template.caption = "Row 7"
hs_template.caption_name = "load"
hs_template.attribute = "Column 2"
hs_template.attribute_name = "hs"
model.input_templates.append(hs_template)

tm10_template = Template()
tm10_template.caption = "Row 7"
tm10_template.caption_name = "load"
tm10_template.attribute = "Column 3"
tm10_template.attribute_name = "tm10"
model.input_templates.append(tm10_template)

dir_template = Template()
dir_template.caption = "Row 7"
dir_template.caption_name = "load"
dir_template.attribute = "Column 4"
dir_template.attribute_name = "dir"
model.input_templates.append(dir_template)

# also template for the output
z2p_template = Template()
z2p_template.caption = ""
z2p_template.attribute = "z-2%"
z2p_template.attribute_name = "z2p"
model.output_templates.append(z2p_template)

### Model validation

In [ ]:
if model.is_valid():
    print ("Model is ok")
else:
    print ("Model is not ok")
    for message in model.validate():
        print (message)

### Reliability analysis

Defining the reliability project:

In [ ]:
project = ReliabilityProject()
project.model = model # reference to the considered model

Printing model input and output:

In [ ]:
print("input parameters:")
for variable in project.variables:
    print(variable)

print("")
print("output parameters:")
for parameter in project.model.output_parameters:
    print(parameter)

Defining the random variables (based on the input parameters):

In [ ]:
project.variables["load.h"].distribution = DistributionType.log_normal
project.variables["load.h"].mean = 2
project.variables["load.h"].variation = 0.1

project.variables["load.hs"].distribution = DistributionType.log_normal
project.variables["load.hs"].variation = 0.2

project.variables["load.tm10"].distribution = DistributionType.log_normal
project.variables["load.tm10"].variation = 0.2

project.variables["load.dir"].distribution = DistributionType.deterministic

Defining the limit state function, $P(z_{2p}>1.5)$:

In [ ]:
project.limit_state_function.compare_type = CompareType.greater_than
project.limit_state_function.parameter = "z2p"
project.limit_state_function.critical_value = 1.5

Choosing calculation method and settings:

In [ ]:
project.settings.reliability_method = ReliabilityMethod.form
project.settings.relaxation_factor = 0.75
project.settings.maximum_iterations = 50

project.settings.max_parallel_processes = 4
project.settings.save_convergence = True
project.settings.save_realizations = True

Running the reliability calculations:

In [ ]:
project.run()
print("calculation completed!")

Getting insight into the results:

In [ ]:
project.design_point.print()
project.design_point.plot_alphas()
project.design_point.plot_convergence()
project.design_point.plot_realizations()

Repeating the reliability calculations with a different calculation method:

In [ ]:
project.settings.reliability_method = ReliabilityMethod.directional_sampling
project.settings.minimum_directions = 100
project.settings.maximum_directions = 200
project.settings.variation_coefficient = 0.02

project.settings.max_parallel_processes = 4
project.settings.save_convergence = True
project.settings.save_realizations = True

project.run()
print("calculation completed!")

project.design_point.print()
project.design_point.plot_alphas()
project.design_point.plot_convergence()
project.design_point.plot_realizations()